In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn as sns

nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagomacedo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thiagomacedo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thiagomacedo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df = pd.read_csv('tweets.csv')
df.head()

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,RT @EdegarPretto O meu candidato vice-governad...,1.0
1,"1,51E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,0,Periga o Argenta chamar o Simon pra responder ...,NaN
2,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,RT @WinstonLing Segue o link para assistirem o...,1.0
3,"7,15E+17",2022-08-08T01:08:20Z,"1,56E+18",pt,88,RT @EdegarPretto Eu quero dizer aos gaúchos e ...,NaN
4,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   author_id      3000 non-null   object 
 1   created_at     3000 non-null   object 
 2   id             3000 non-null   object 
 3   lang           3000 non-null   object 
 4   retweet_count  3000 non-null   int64  
 5   text           3000 non-null   object 
 6   sinal          1130 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB


In [14]:
df = df[['text', 'sinal']]
df.head()

,text,sinal
0,RT @EdegarPretto O meu candidato vice-governad...,1.0
1,Periga o Argenta chamar o Simon pra responder ...,NaN
2,RT @WinstonLing Segue o link para assistirem o...,1.0
3,RT @EdegarPretto Eu quero dizer aos gaúchos e ...,NaN
4,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0.0


In [15]:
df = df.dropna(axis=0)
df.head()

,text,sinal
0,RT @EdegarPretto O meu candidato vice-governad...,1.0
2,RT @WinstonLing Segue o link para assistirem o...,1.0
4,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0.0
5,@EduardoLeite_ 🤡,0.0
6,Nós já governamos o Rio Grande Sul com @olivio...,0.0


In [16]:
df.sinal = df.sinal.astype('int64')
df.head()

,text,sinal
0,RT @EdegarPretto O meu candidato vice-governad...,1
2,RT @WinstonLing Segue o link para assistirem o...,1
4,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0
5,@EduardoLeite_ 🤡,0
6,Nós já governamos o Rio Grande Sul com @olivio...,0


In [17]:
df.shape

(1130, 2)

In [18]:
df.duplicated().sum()

232

In [19]:
df = df.drop_duplicates()
df.head()

,text,sinal
0,RT @EdegarPretto O meu candidato vice-governad...,1
2,RT @WinstonLing Segue o link para assistirem o...,1
4,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0
5,@EduardoLeite_ 🤡,0
6,Nós já governamos o Rio Grande Sul com @olivio...,0


In [20]:
df.shape

(898, 2)

In [22]:
df = df.reset_index(drop=True)
df.head()

,text,sinal
0,RT @EdegarPretto O meu candidato vice-governad...,1
1,RT @WinstonLing Segue o link para assistirem o...,1
2,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0
3,@EduardoLeite_ 🤡,0
4,Nós já governamos o Rio Grande Sul com @olivio...,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    898 non-null    object
 1   sinal   898 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.2+ KB


In [26]:
df = df.rename(columns={'text':'tweet', 'sinal':'positive'})
df.head()

,tweet,positive
0,RT @EdegarPretto O meu candidato vice-governad...,1
1,RT @WinstonLing Segue o link para assistirem o...,1
2,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0
3,@EduardoLeite_ 🤡,0
4,Nós já governamos o Rio Grande Sul com @olivio...,0


In [28]:
def tokenize(text):
    # make all text lowercase 
    text = text.lower()

    # remove all type of punctuations 
    text = re.sub(f'[^a-z0-9]',' ', text)

    # tokenize the text
    tokens = word_tokenize(text)
    
    # lemmatize the text
    tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    # remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    return tokens